<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
! pip install git+https://github.com/yeoun9/torchpm.git
! pip install sympytorch
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.estimated_parameter import *
import torch as tc
import numpy as np
import matplotlib.pyplot as plt

  Cloning https://github.com/yeoun9/torchpm.git to c:\users\sungwoo\appdata\local\temp\pip-req-build-6ovpf8fq
  Resolved https://github.com/yeoun9/torchpm.git to commit f0fc85b46ba8f318c41e03956238c29ba9cd5205
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/yeoun9/torchpm.git 'C:\Users\sungwoo\AppData\Local\Temp\pip-req-build-6ovpf8fq'


# Model Building

In [41]:
tc.set_default_dtype(tc.float64)

#TODO 주의사항 다운로드 할때 캐쉬 파일이 있으면 업데이트가 되지 않아서 이상하게 계산되는 경우가 있으니 주의해야함.
dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO.csv'
# dataset_file_path = './THEO.csv'
column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', 'MDV', 'RATE', 'BWT']

#TODO CPU랑 GPU랑 차이남. 근데 float64나 float32를 기본으로 설정해서 똑같이 나오도록 유도
device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
# device = tc.device("cpu")
dataset = CSVDataset(dataset_file_path, column_names, device)

class BasementModel(predfunction.PredictionFunctionByTime) :
    def __init__(self, dataset: tc.utils.data.Dataset, column_names: Iterable[str], output_column_names: Iterable[str], *args, **kwargs):
        super().__init__(dataset, column_names, output_column_names, *args, **kwargs)

        self.theta_0 = Theta(1.5, 0, 10)
        self.theta_1 = Theta(32, 0, 50)
        self.theta_2 = Theta(0.08, 0, 1)

        self.eta_0 = Eta()
        self.eta_1 = Eta()
        self.eta_2 = Eta()

        self.eps_0 = Eps()
        self.eps_1 = Eps()

        self.gut_model = linearode.Comp1GutModelFunction()

        self.initialize()
    
    def _calculate_parameters(self, **para):
        k_a = self.theta_0()*tc.exp(self.eta_0())
        #TODO
        v = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
        k_e = self.theta_2()*tc.exp(self.eta_2())
        para['AMT'] = tc.tensor(320., device=self.dataset.device)
        return para | {"k_a": k_a, "v": v, "k_e": k_e}

    def _calculate_preds(self, t, amt, rate, **para) -> tc.Tensor:
        dose = amt
        k_a = para['k_a']
        v = para['v']
        k_e = para['k_e']
        # return (dose / v * k_a) / (k_a - k_e) * (tc.exp(-k_e*t) - tc.exp(-k_a*t))
        ############ Sympy Version Function #############
        comps = self.gut_model(t, k_a, k_e, dose)
        return comps[1]/v
        
    def _calculate_error(self, y_pred, **para) -> tc.Tensor:
        return y_pred +  y_pred * self.eps_0() + self.eps_1(), para

pred_function_module = BasementModel(dataset = dataset,
                            column_names = column_names,
                            output_column_names=['ID'],)

omega = Omega([tc.tensor([0.4397,
                        0.0575,  0.0198, 
                        -0.0069,  0.0116,  0.0205], device = device)], [False])
sigma = Sigma( [tc.tensor([0.0177, 0.0762], device = device)], [True])

model = models.FOCEInter(pred_function_module, 
                        theta_names=['0', '1', '2'],
                        eta_names=[['0', '1','2']], 
                        eps_names= [['0','1']], 
                        omega=omega, 
                        sigma=sigma)

model = model.to(device)

#Model Fitting

In [42]:
model.fit_population(learning_rate = 1, tolerance_grad = 1e-5, tolerance_change= 1e-5)

running_time :  0.7493245601654053 	 total_loss: tensor(566.2649, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  1.3314123153686523 	 total_loss: tensor(329.6858, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  1.9162180423736572 	 total_loss: tensor(264.6675, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  2.4984073638916016 	 total_loss: tensor(197.2443, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.075737237930298 	 total_loss: tensor(186.1239, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.6533355712890625 	 total_loss: tensor(173.8079, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  4.227323770523071 	 total_loss: tensor(171.3669, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  4.814727306365967 	 total_loss: tensor(158.5640, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.399420976638794 	 total_loss: tensor(135.5841, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.98532509803772 	 total_

FOCEInter(
  (pred_function_module): BasementModel(
    (theta_0): Theta()
    (theta_1): Theta()
    (theta_2): Theta()
    (eta_0): Eta(
      (parameter_values): ParameterDict(
          (1): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (2): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (3): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (4): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (5): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (6): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (7): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (8): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (9): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (10): Parameter containing: [torch.cuda.DoubleTensor of size  (GPU 0)]
          (11): Par

# Result

In [54]:
eval_result = model.descale().evaluate()

print('time-pred')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
for id, time_data in eval_result['times'].items() :
    y_pred = eval_result['preds'][id]
    ax.plot(time_data.to('cpu'), y_pred.detach().to('cpu').numpy())
plt.show()

print('time-cwres')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
for id, time_data in eval_result['times'].items() :
    cwres_value = eval_result['cwress'][id]
    ax.plot(time_data.masked_select(eval_result['mdv_masks'][id]).to('cpu'), cwres_value.detach().to('cpu').numpy())
plt.show()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [44]:
for k, v in eval_result.items():
    print(k)
    print(v)

total_loss
tensor(92.1253, device='cuda:0')
losses
{'1': 20.09432054779129, '2': 17.494065704837382, '3': -1.2956903537397277, '4': 9.294098169619797, '5': 20.158547903457574, '6': 4.118904719513731, '7': 1.2929950559537637, '8': 3.9580997252746535, '9': 2.977555221370233, '10': 4.822593456769802, '11': -2.4200388653017093, '12': 11.629896902603614}
times
{'1': tensor([ 0.0000,  0.0000,  0.2500,  0.5700,  1.1200,  2.0200,  3.8200,  5.1000,
         7.0300,  9.0500, 12.1200, 24.3700], device='cuda:0',
       dtype=torch.float32), '2': tensor([ 0.0000,  0.0000,  0.2700,  0.5200,  1.0000,  1.9200,  3.5000,  5.0200,
         7.0300,  9.0000, 12.0000, 24.3000], device='cuda:0',
       dtype=torch.float32), '3': tensor([ 0.0000,  0.0000,  0.2700,  0.5800,  1.0200,  2.0200,  3.6200,  5.0800,
         7.0700,  9.0000, 12.1500, 24.1700], device='cuda:0',
       dtype=torch.float32), '4': tensor([ 0.0000,  0.0000,  0.3500,  0.6000,  1.0700,  2.1300,  3.5000,  5.0200,
         7.0200,  9.0200, 11

In [45]:
cov_result = model.descale().covariance_step()
print(cov_result)

id 1
id 2
id 3
id 4
id 5
id 6
id 7
id 8
id 9
id 10
id 11
id 12
{'cov': tensor([[ 3.5608e-03,  2.6993e-02, -1.5673e-04,  9.9913e-04,  8.2931e-05,
          9.8599e-05, -7.3628e-07,  1.8294e-04,  3.3704e-04, -3.8012e-04,
          3.2504e-03],
        [ 2.6993e-02,  3.4340e-01, -1.4778e-03,  1.3187e-03, -3.4414e-03,
          1.6031e-04, -3.1016e-03,  1.3220e-03,  2.5405e-03, -1.1178e-03,
          1.0364e-02],
        [-1.5673e-04, -1.4778e-03,  7.7138e-06, -9.8072e-06,  6.7894e-06,
         -3.3161e-06,  4.9767e-06, -8.5714e-06, -1.5218e-05,  1.4079e-05,
         -1.1653e-04],
        [ 9.9913e-04,  1.3187e-03, -9.8072e-06,  2.5533e-02,  3.2433e-03,
          2.3392e-04, -3.5577e-04, -2.2654e-04, -3.5223e-04, -1.8353e-04,
         -2.5278e-03],
        [ 8.2931e-05, -3.4414e-03,  6.7894e-06,  3.2433e-03,  6.3207e-04,
          6.4848e-05,  2.0963e-04, -1.9151e-05, -6.6566e-05, -4.6176e-05,
         -1.5864e-04],
        [ 9.8599e-05,  1.6031e-04, -3.3161e-06,  2.3392e-04,  6.4848e-05,


In [46]:
for (name, para) in model.to('cpu').named_parameters():
    print(name)
    print(para.detach().numpy())

pred_function_module.theta_0.parameter_value
1.4903191940893723
pred_function_module.theta_1.parameter_value
32.46531960552304
pred_function_module.theta_2.parameter_value
0.08726725546527295
pred_function_module.eta_0.parameter_values.1
-0.09695220144893717
pred_function_module.eta_0.parameter_values.2
0.34226750775831866
pred_function_module.eta_0.parameter_values.3
0.4257257301188131
pred_function_module.eta_0.parameter_values.4
-0.3396795174421511
pred_function_module.eta_0.parameter_values.5
-0.025455512759810485
pred_function_module.eta_0.parameter_values.6
-0.4662754004253999
pred_function_module.eta_0.parameter_values.7
-0.8582128169233869
pred_function_module.eta_0.parameter_values.8
-0.05623675701512883
pred_function_module.eta_0.parameter_values.9
1.391493465428994
pred_function_module.eta_0.parameter_values.10
-0.7090186135372448
pred_function_module.eta_0.parameter_values.11
0.8910585471605555
pred_function_module.eta_0.parameter_values.12
-0.4984640684364717
pred_function

#Simulation

In [53]:
tc.manual_seed(42)
dataset = CSVDataset(dataset_file_path, column_names, device=tc.device('cpu'))
simulation_result = model.to('cpu').simulate(dataset, 300)

for id, time_data in simulation_result['times'].items() :
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    print('id', id)
    
    p95 = np.percentile(tc.stack(simulation_result['preds'][id]), 95, 0)
    p50 = np.percentile(tc.stack(simulation_result['preds'][id]), 50, 0)
    average = np.average(tc.stack(simulation_result['preds'][id]), 0)
    p5 = np.percentile(tc.stack(simulation_result['preds'][id]), 5, 0)
    
    ax.plot(time_data.to('cpu'), p95, color="black")
    ax.plot(time_data.to('cpu'), p50, color="green")
    ax.plot(time_data.to('cpu'), average, color="red")
    ax.plot(time_data.to('cpu'), p5, color="black")
    
    for y_pred in simulation_result['preds'][id] :
        ax.plot(time_data.to('cpu'), y_pred.detach().to('cpu'), marker='.', linestyle='', color='gray')
    plt.show()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!